In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import torch

import warnings
warnings.filterwarnings("ignore")

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

from openai_assistant.ai_reply import OpenAIClient
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
data = pd.read_csv('facebook_data\留言檔_已清理\徐巧芯留言.csv')

In [6]:
data['attitude'] = ""
data[["text", "attitude"]].head()

,text,attitude
0,加油,
1,空戰天后Go Go Fighting ❣️,
2,加油,
3,馬文君委員加油！巧芯加油！,
4,辛苦了！加油,


In [8]:
client = OpenAIClient()

template = """There are some comments from Facebook page of politician called {name}. 
            Help me decide whether the comment is positive or negative to this politician and 國民黨 (KMT) party. 
            If it is positive, please reply with 1. If it is negative, please reply with 0. 
            Please do not reply with any other number or word. The comment is as follows: {comments}"""

i = 0

for index, row in data.iterrows():
    if i > 1000:
        break
    else:
        i += 1
        reply = await client.text_reply(template.format(name="徐巧芯", comments=row['text'])) 
        data.at[index, 'attitude'] = reply
        #print(row['text'], ": ", data.at[index, 'attitude'])
        #print(row['text'], ": ", reply)

In [9]:
# Filter out the rows that have been labeled
label_data = data[data['attitude'] != '']
label_data.to_csv('facebook_data\label_comment\labeled.csv', index=False)

In [ ]:
# Train XGBoost model
data = data[data['attitude'].isin(['0', '1'])]
X = np.array(data['bert_embedding'].tolist())
y = data['attitude'].astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")